In [ ]:
%%time
import pandas as pd
import numpy as np
import ml_metrics as metrics
from sklearn import cross_validation
from sklearn import ensemble
from sklearn.decomposition import RandomizedPCA
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import SelectKBest, RFECV, SelectFromModel
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
import xgboost as xgb
from datetime import datetime

In [ ]:
%%time
temp_train_2014_1 = pd.read_csv('CAX_Train_2014_Jan_to_Jun.csv')
temp_train_2014_2 = pd.read_csv('CAX_Train_2014_Jul_to_Dec.csv')
temp_train_2015 = pd.read_csv('CAX_Train_2015.csv')
testing = pd.read_csv('CAX_TestSet.csv')
training = pd.concat([temp_train_2014_1, temp_train_2014_2, temp_train_2015], ignore_index=True)
# orig_weather = pd.read_csv('weather.csv')
#dest_weather = orig_weather.copy(deep=True)
# codes = pd.read_csv('airport-codes.csv')
# airports = pd.read_csv('airports.csv')

In [ ]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [ ]:
test = pd.DataFrame(training['ORIGIN'].values).drop_duplicates()

In [ ]:
codes_iata = pd.DataFrame(codes[['iata_code', 'gps_code']]).drop_duplicates('iata_code')
codes_iata = codes_iata.dropna()
codes_iata = codes_iata[codes_iata.iata_code != '0']
print len(codes_iata)

In [ ]:
training_test = pd.merge(training, codes_iata, left_on='ORIGIN', right_on='iata_code')

In [ ]:
print len(training), len(training_test)

In [ ]:
orig_weather.head()

In [ ]:
test_hour = [str((i/100) * 100) if (i % 100) < 50 else str((i/100) * 100 + 100) for i in training['CRS_DEP_TIME'].values]

In [ ]:
%%time
training['DEP_HOUR'] = [str((i/100) * 100) if (i % 100) < 30 else str((i/100) * 100 + 100) for i in training['CRS_DEP_TIME'].values]
training['ARR_HOUR'] = [str((i/100) * 100) if (i % 100) < 30 else str((i/100) * 100 + 100) for i in training['CRS_ARR_TIME'].values]

In [ ]:
%%time
orig_weather.columns = ['ORIG_DATE', 'ORIG_TIME', 'ORIG_TIME_BLK', 'ORIG_CITY', 'ORIG_STATE', 'ORIG_ZIP', 'ORIG_AIRPORT_CODE', 'ORIG_TIME_CST', 'ORIG_DATE_UTC', 'ORIG_TEMPERATURE_F', 'ORIG_DEW_POINT_F', 'ORIG_HUMIDITY', 'ORIG_SEA_LEVEL_PRESSURE_IN', 'ORIG_VISIBILITY_MPH', 'ORIG_WIND_DIRECTION', 'ORIG_WIND_SPEED_MPH', 'ORIG_GUST_SPEED_MPH', 'ORIG_PRECIPITATION_IN', 'ORIG_EVENTS', 'ORIG_CONDITIONS', 'ORIG_WIND_DIR_DEGREES']
#dep_weather.columns = ['DEST_DATE', 'DEST_TIME', 'DEST_TIME_BLK', 'DEST_CITY', 'DEST_STATE', 'DEST_ZIP', 'DEST_AIRPORT_CODE', 'DEST_TIME_CST', 'DEST_DATE_UTC', 'DEST_TEMPERATURE_F', 'DEST_DEW_POINT_F', 'DEST_HUMIDITY', 'DEST_SEA_LEVEL_PRESSURE_IN', 'DEST_VISIBILITY_MPH', 'DEST_WIND_DIRECTION', 'DEST_WIND_SPEED_MPH', 'DEST_GUST_SPEED_MPH', 'DEST_PRECIPITATION_IN', 'DEST_EVENTS', 'DEST_CONDITIONS', 'DEST_WIND_DIR_DEGREES']

In [ ]:
%%time
test_weather = pd.merge(orig_weather, codes[['gps_code', 'iata_code']], left_on='ORIG_AIRPORT_CODE', right_on='gps_code')
#arr_weather = pd.merge(orig_weather, codes[['gps_code', 'local_code']], left_on='ORIG_AIRPORT_CODE', right_on='gps_code')
#dep_weather = pd.merge(dest_weather, codes[['gps_code', 'local_code']], left_on='DEST_AIRPORT_CODE', right_on='gps_code')

In [ ]:
# %%time
# training_ww = pd.merge(training, test_weather, left_on=['FL_DATE', 'ORIGIN','ARR_TIME_BLK'], right_on=['ARR_DATE', 'local_code', 'ARR_TIME_BLK'])
# training_ww = pd.merge(training_ww, dep_weather, left_on=['FL_DATE', 'DEST','DEP_TIME_BLK'], right_on=['DEP_DATE', 'local_code', 'DEP_TIME_BLK'])

In [ ]:
# origin_state = pd.DataFrame(training.ORIGIN_STATE_ABR.drop_duplicates())
# dest_state = pd.DataFrame(training.DEST_STATE_ABR.drop_duplicates())
# print len(origin_state), len(dest_state)

In [ ]:
# arr_codes = arr_weather.ARR_AIRPORT_CODE.drop_duplicates()

In [ ]:
# %%time
# training_nw = training[~(training.id.isin(training_ww.id))]

In [ ]:
# print len(training), (len(training_ww) + len(training_nw))

In [ ]:
# %%time
# with_dest_match = pd.merge(all_train, destinations_df)

In [ ]:
%%time
training_imputed = DataFrameImputer().fit_transform(training)

In [ ]:
%%time
training_imputed.UNIQUE_CARRIER = le.fit_transform(training_imputed.UNIQUE_CARRIER)
training_imputed.ORIGIN = le.fit_transform(training_imputed.ORIGIN)
training_imputed.ORIGIN_CITY_NAME = le.fit_transform(training_imputed.ORIGIN_CITY_NAME)
training_imputed.ORIGIN_STATE_ABR = le.fit_transform(training_imputed.ORIGIN_STATE_ABR)
training_imputed.DEST = le.fit_transform(training_imputed.DEST)
training_imputed.DEST_CITY_NAME = le.fit_transform(training_imputed.DEST_CITY_NAME)
training_imputed.DEST_STATE_ABR = le.fit_transform(training_imputed.DEST_STATE_ABR)

In [ ]:
%%time
features = [c for c in training.columns if c not in ['FL_DATE', 'FL_NUM', 'TAIL_NUM', 'DEP_TIME_BLK', 'ARR_TIME_BLK', 'ARR_DEL15', 'id']]

In [ ]:
%%time
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(training_imputed[features], training_imputed['ARR_DEL15'], test_size=0.30)

In [ ]:
# %%time
# xgb_train = xgb.DMatrix(features_train.values, label=labels_train.values)

In [ ]:
%%time
gbm = xgb.XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27).fit(features_train.values, labels_train.values)
# forest = ensemble.RandomForestClassifier(min_samples_split=50, n_estimators=20, n_jobs=-1).fit(features_train, labels_train)

In [ ]:
%%time
pred = gbm.predict(features_test)
# pred = forest.predict(features_test)

In [ ]:
%%time
print accuracy_score(pred, labels_test)

In [ ]:
%%time
testing.UNIQUE_CARRIER = le.fit_transform(testing.UNIQUE_CARRIER)
testing.ORIGIN = le.fit_transform(testing.ORIGIN)
testing.ORIGIN_CITY_NAME = le.fit_transform(testing.ORIGIN_CITY_NAME)
testing.ORIGIN_STATE_ABR = le.fit_transform(testing.ORIGIN_STATE_ABR)
testing.DEST = le.fit_transform(testing.DEST)
testing.DEST_CITY_NAME = le.fit_transform(testing.DEST_CITY_NAME)
testing.DEST_STATE_ABR = le.fit_transform(testing.DEST_STATE_ABR)

In [ ]:
%%time
test_pred = pd.DataFrame(forest.predict(testing[features]))

In [ ]:
%%time
submission = pd.DataFrame()
submission['id'] = testing['id']
submission['ARR_DEL15'] = test_pred

In [ ]:
print datetime.now()

In [ ]:
filename = 'submission' + str(datetime.now().strftime("%Y-%m-%d-%H%M%S")) + '.csv'

In [ ]:
%%time
submission.to_csv(filename, index=False)